In [1]:
import time, datetime

In [2]:
datetime.datetime.now()

datetime.datetime(2018, 2, 26, 20, 52, 48, 899579)

In [3]:
MAX_FEATURES = None
NB = True#False
CLF = 'SVC'#'LR'
SUB_FILE = 'submissions/sub_{}_{}_{}_{}.csv'.format(MAX_FEATURES, NB, CLF, int(time.time()))
SUB_FILE

'submissions/sub_None_True_SVC_1519678370.csv'

In [4]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [5]:
%matplotlib inline

In [6]:
train = pd.read_csv('../../toxic/data/train.csv')

In [7]:
test = pd.read_csv('../../toxic/data/test.csv')

In [8]:
subm = pd.read_csv('../../toxic/data/sample_submission.csv')

In [9]:
train.shape

(159571, 8)

In [10]:
test.shape

(153164, 2)

In [11]:
train.comment_text.fillna('unknown', inplace=True)

In [12]:
test.comment_text.fillna('unknown', inplace=True)

In [13]:
import string
special_punc = '“”¨«»®´·º½¾¿¡§£₤‘’'
all_punc = special_punc+string.punctuation

# replace punctuations

In [14]:
train.comment_text = train.comment_text.str.replace('[{}]'.format(all_punc),'')

In [15]:
test.comment_text = test.comment_text.str.replace('[{}]'.format(all_punc),'')

In [16]:
train.comment_text[0]

'Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted They werent vandalisms just closure on some GAs after I voted at New York Dolls FAC And please dont remove the template from the talk page since Im retired now892053827'

In [17]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, strip_accents='unicode', sublinear_tf=True, max_features=MAX_FEATURES)

In [18]:
train_term_doc = vec.fit_transform(train.comment_text)

In [19]:
test_term_doc = vec.transform(test.comment_text)

In [20]:
train_term_doc, test_term_doc

(<159571x394451 sparse matrix of type '<class 'numpy.float64'>'
 	with 13629342 stored elements in Compressed Sparse Row format>,
 <153164x394451 sparse matrix of type '<class 'numpy.float64'>'
 	with 10975331 stored elements in Compressed Sparse Row format>)

In [21]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [23]:
import pdb

In [24]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import svm
from scipy import sparse
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        pdb.set_trace()
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        y = y.values
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        pdb.set_trace()
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        #self._clf = svm.SVC(probability=True).fit(x_nb, y)
        return self
    
    def feature_importance(self):
        return self._clf.feature_importance

In [25]:
x = train_term_doc
test_x = test_term_doc

In [26]:
label_cols = list(train.columns[-6:])
label_cols

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
%%time
preds = np.zeros((len(test), len(label_cols)))

for i, label in enumerate(label_cols):
    print('fit', label)
    model = NbSvmClassifier(C=4, dual=True, n_jobs=-1).fit(x, train[label])
    preds[:,i] = model.predict_proba(test_x)[:,1]

fit toxic
> <ipython-input-24-5a89a7bef023>(35)fit()
-> x_nb = x.multiply(self._r)
(Pdb) _r.shape
*** NameError: name '_r' is not defined
(Pdb) self._r.shape
(1, 394451)
(Pdb) n
> <ipython-input-24-5a89a7bef023>(36)fit()
-> self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
(Pdb) x.shpae
*** AttributeError: shpae not found
(Pdb) x.shape
(159571, 394451)
(Pdb) x_nb.shape
(159571, 394451)
(Pdb) c


/home/kai/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


> <ipython-input-24-5a89a7bef023>(22)predict_proba()
-> return self._clf.predict_proba(x.multiply(self._r))
(Pdb) x.shape
(153164, 394451)
(Pdb) self._r.shape
(1, 394451)


preds = np.zeros((len(test), len(label_cols)))

for i, label in enumerate(label_cols):
    print('fit', label)
    model = LogisticRegression(C=4, dual=True, n_jobs=-1).fit(x, train[label])
    preds[:,i] = model.predict_proba(test_x)[:,1]

In [ ]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv(SUB_FILE, index=False)

In [ ]:
submission.shape

In [28]:
submission.shape

(153164, 7)

In [29]:
submission.head(2)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999973,0.049945,0.999761,0.004672,0.854442,0.256823
1,0000247867823ef7,0.001196,0.000346,0.000727,0.000084,0.001928,0.000386


import lightgbm as lgb

params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 200,
        'learning_rate': 0.1,
        'num_leaves': 127,
        'num_threads': 18, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 300,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 1.0, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'bagging_fraction': 1.0, #Randomly select part of data without resampling
        'bagging_freq': 0, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'metric' : 'auc'
    }

model_1 = lgb.train(params, train_set=lgb_train_1, valid_sets=lgb_train_1, verbose_eval=5)

lgb_train_1 = lgb.Dataset(train_term_doc, train.toxic.values)

p_1 = model_1.predict(test_term_doc)

submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols[0])], axis=1)
#submission.to_csv('submission1.csv', index=False)